### What is Project Euler?

https://en.wikipedia.org/wiki/Project_Euler  
https://projecteuler.net/about

### Timer:
One of the key points here in Project Euler is that the answer should be worked out in a minute with the code. Since I have (hopefully) lots of code snippets to run and test, I define my own funtion to print the execution result and running time. 

In [1]:
import time

def test(solution, *args):
    start_time = time.time()
    print('Answer:', solution(args))
    end_time = time.time()
    print('Runtime:', start_time-end_time, 'seconds')

In [2]:
import timeit

def timeit_and_print(solution, *args, **kwargs):
    
    def wrapper(func, *args, **kwargs):
        def wrapped():
            return func(*args, **kwargs)
        return wrapped

    wrapped = wrapper(solution, *args, **kwargs)
    
    print('Result:\t', solution(*args))
    print('Time:\t', timeit.timeit(wrapped, number=1))

### Problem 1: Multiples of 3 and 5
>If we list all the natural numbers below 10 that are multiples of 3 or 5, we get 3, 5, 6 and 9. The sum of these multiples is 23.  
Find the sum of all the multiples of 3 or 5 below 1000.

First idea: find multiples of 3 and multiple of 5 respectively, find their union, calculate the sum.

In [3]:
def Problem1(n):
    multiple_of_3s = [i for i in range(n) if not i%3]
    multiple_of_5s = [i for i in range(n) if not i%5]
    set_union = set(multiple_of_3s).union(multiple_of_5s)
    return(sum(set_union))

timeit_and_print(Problem1,1000)

Result:	 233168
Time:	 0.0003464999999778229


Actually we don't really need to use set. A list and list comprehension do the work.  

In [4]:
def Problem1_Sol2(n):
    return (sum(i for i in range(n) if not (i%3 and i%5)))
    
timeit_and_print(Problem1_Sol2,1000)

Result:	 233168
Time:	 0.00011999999969702912


To explain the conditional expression:
1. What we need is the numbers either divided by 3 or divided by 5, which satisfy `(i%3==0 or i%5==0)`
2. The `x==0` boolean expression is equivalent to `not x`
3. Basic boolean logic: `not A or not B` is equivalent to `not (A and B)`

In fact, we don't really need to iterate from 1 to 1000. Sum of (finite) series and a little skill are enough to accomplish the task in O(1) time instead of O(n). 

In [5]:
import math

def Problem1_Sol3(max):
    
    def sum_of_multiples(p):
        n = math.ceil(max/p) - 1
        return n*p*(n+1)/2      
        
    return sum_of_multiples(3) + sum_of_multiples(5) - sum_of_multiples(15)

timeit_and_print(Problem1_Sol3, 1000)    

Result:	 233168.0
Time:	 4.800000169780105e-06


### Problem 2: (Sum of) Even Fibonacci numbers
> Each new term in the Fibonacci sequence is generated by adding the previous two terms. By starting with 1 and 2, the first 10 terms will be:  
1, 2, 3, 5, 8, 13, 21, 34, 55, 89, ...  
By considering the terms in the Fibonacci sequence whose values do not exceed four million, find the sum of the even-valued terms.

There is nothing hard about the algorithm for this problem, but using a [Generator](https://wiki.python.org/moin/Generators) here could save you tons of time and space

In [6]:
def Problem2(m):
    
    def Fibo():
        a,b = 1,2
        while a < max:
            if not a % 2:
                yield a
            a, b = b, a + b
    
    max = m
    return sum(Fibo())

timeit_and_print(Problem2, 4*10**6)

Result:	 4613732
Time:	 7.599999662488699e-06


### Problem 3: Largest Prime Factor
>The prime factors of 13195 are 5, 7, 13 and 29.  
What is the largest prime factor of the number 600851475143 ?

When looking at *the big number*, it must has one or multiple prime factors. If we **recurrently / completely** divide it with all the numbers that is not bigger than it, the last factor would be the answer.

In [23]:
def Problem3(num):
    best = 1
    i = 2
    while num != 1 and i <= num:
        while not num % i:
            best = i
            num /= i
        i += 1
    return best

timeit_and_print(Problem3, 600851475143)

Result:	 6857
Time:	 0.001595699999597855


A place to improve is that we can limit the divisor to the square root of the big number. If it remains to be 1, then the last factor is what we want; if not, the the remaining number is the answer.

In [26]:
import math

def Problem3_1(num):
    sqroot = math.sqrt(num)
    i = 2
    best = 1
    while i <= sqroot:
        while not num % i:
            best = i
            num /= i
        i += 1
    return best if num == 1 else num

timeit_and_print(Problem3_1, 600851475143)

Result:	 6857
Time:	 0.13416380000126082


Another trick here is that, we can get rid of the composite numbers from the divisor list by avoiding all the even numbers except 2. This reduces half of the length, leaving `2, 3, 5, 7, 9, 11, 13, 15, 17 ...`.

If we dig a little deeper, we can avoid the composite numbers `9, 15, 21 ...` (in forms of `6k+3`) by changing the step size from constant 2 (`5,7,9,11,13,15...`) to alternating 2-and-4 (`5,7,11,13,17,19...`). To convert between 2 and 4, the idea of bit calculation `step ^= 6` (or`step = step XOR 110` in binary) is so clever.

In [27]:
def Problem3_2(num):
    
    def possible_prime_factors(num):
        yield 2
        yield 3
        fac = 5
        step = 2
        while fac * fac <= num:
            yield fac
            fac += step
            step ^= 6
                
    assert num>2
    
    best = None
    for factor in possible_prime_factors(num):
        while not num % factor:
            best = factor
            num /= factor
    return best if num == 1 else num

timeit_and_print(Problem3_2, 600851475143)

Result:	 6857
Time:	 0.07169570000041858


Although it takes longer (on this test case) than the very original solution, the idea is just genius.

### Problem 4: Largest palindrome product
> A palindromic number reads the same both ways. The largest palindrome made from the product of two 2-digit numbers is 9009 = 91 × 99.
Find the largest palindrome made from the product of two 3-digit numbers.

First we need a function to determine if a number is a palindrome, and some Python string trick can do the job. 

To search the largest product, iterate `a` from 999 to 100 and iterate `b` from 999 to `a`. If, at any time, we find that the product is smaller than the current `max_palin` or we update the `max_palin`, we just skip for this combination of `a` and `b` because it certainly won't give us any bigger product.

In [34]:
def Problem4():
    
    max_palin = 0

    def is_palin(num):
        reversed = int(str(num)[::-1])
        return num == reversed

    for a in range(999,99,-1):
        for b in range(999,a,-1):
            product = a * b
            if product < max_palin:
                break
            if is_palin(product):
                max_palin = product
                break

    return max_palin

timeit_and_print(Problem4)

Result:	 906609
Time:	 0.002409599999737111
